<a href="https://colab.research.google.com/github/itsabbhii/Training-AIML/blob/main/NLP/NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/itsabbhii/Training-AIML.git
%cd Training-AIML/NLP/

Cloning into 'Training-AIML'...
remote: Enumerating objects: 1136, done.
remote: Counting objects: 100% (934/934), done.
remote: Compressing objects: 100% (913/913), done.
remote: Total 1136 (delta 54), reused 858 (delta 14), pack-reused 202 (from 1)
Receiving objects: 100% (1136/1136), 54.22 MiB | 9.05 MiB/s, done.
Resolving deltas: 100% (128/128), done.
Updating files: 100% (942/942), done.
/content/Training-AIML/NLP


In [2]:
import nltk,string,pickle
import random
from nltk.corpus import movie_reviews,stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

In [13]:
nltk.download(['movie_reviews', 'stopwords', 'wordnet'])
documents = [
    (list(movie_reviews.words(fileid)), category)
    for category in movie_reviews.categories()
    for fileid in movie_reviews.fileids(category)
]
random.shuffle(documents)
reviews, sentiments = zip(*documents)

[nltk_data] Downloading package movie_reviews to /root/nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [14]:
def preprocess(review):
  lemmatizer = WordNetLemmatizer()
  stopword = set(stopwords.words('english'))
  return ''.join([lemmatizer.lemmatize(word.lower()) for word in review if word not in string.punctuation and word.lower() not in stopword])
processed_reviews = [preprocess(review) for review in reviews]

In [15]:
X_train, X_test, y_train, y_test = train_test_split(processed_reviews, sentiments, test_size=0.2)

In [16]:
vectorizer = TfidfVectorizer(max_features=5000)
X_train_vectorized = vectorizer.fit_transform(X_train)
X_test_vectorized = vectorizer.transform(X_test)
model = LogisticRegression().fit(X_train_vectorized, y_train)
y_pred = model.predict(X_test_vectorized)

accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)
print(f"Accuracy: {accuracy}")
print(f"Report: {report}")

Accuracy: 0.4875
Report:               precision    recall  f1-score   support

         neg       0.49      0.98      0.65       195
         pos       0.50      0.02      0.04       205

    accuracy                           0.49       400
   macro avg       0.49      0.50      0.34       400
weighted avg       0.49      0.49      0.34       400



In [17]:
with open("sentiment_model.pkl", "wb") as m, open("tfidf_vectorize.pkl", "wb") as v:
  pickle.dump(model, m)
  pickle.dump(vectorizer, v)

In [9]:
import pickle, string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

with open("sentiment_model.pkl", "rb") as m, open("tfidf_vectorize.pkl", "rb") as v:
  model = pickle.load(m)
  vectorizer = pickle.load(v)


In [11]:
def preprocess_input(input_text):
  lemmatizer = WordNetLemmatizer()
  stopword = set(stopwords.words('english'))
  input_text = input_text.lower()
  input_text = ''.join([lemmatizer.lemmatize(word) for word in input_text if word not in string.punctuation and word not in stopword])
  words = input_text.split()
  words = [word for word in words if len(word) > 1]
  words = [lemmatizer.lemmatize(word) for word in words]
  return ' '.join(words)

In [12]:
custom_review = "i enjoyed it"
preprocessed_review = preprocess_input(custom_review)
vectorized_review = vectorizer.transform([preprocessed_review])
prediction = model.predict(vectorized_review)
print(f"Predicted Sentiment: {prediction[0]}")

Predicted Sentiment: neg
